In [ ]:
# -*- coding: utf-8 -*-
"""
The following notebook uses a dataset downloaded from https://www.dukascopy.com/swiss/english/marketwatch/historical/

@author: Malhar
We are creating a new Datasheet which will be using in our future projects
looking at regressions, and Tree Algorithims
"""
import pandas as pd
pd.set_option('display.max_columns', 500)
df = pd.read_csv("EURUSD_1H.csv")
#print(df.head(10))

In [ ]:
#Next we convert the local time column into a date_time, and the rest to floats
'''
#df['Local time'] = pd.to_datetime(df['Local time'], errors = 'coerce')
doesn't work. The parser failed to understand the column. So we have to do it manually
first we slice the string to remove the excess,
then we convert to date time, using dayfirst. Lastly we rename for convenience
'''
df['Local time'] = df['Local time'].str.slice(0,19)
df['Local time'] = pd.to_datetime(df['Local time'], errors = 'coerce', dayfirst=True)
df = df.rename({'Local time':'Date_Time'}, axis = 1)
#print(df.info())
#we can see all the datatypes are now correct

In [ ]:
#now we need to add potentially useful columns as our dependant variables
#First, lets make Day and month specific columns
df['Month'] = df['Date_Time'].dt.month
df['Day'] = df['Date_Time'].dt.day
df['Hour'] = df['Date_Time'].dt.hour

In [ ]:
#next, the change in the current candle and the range of the current candle
df['Change'] = 10000 * (df['Close']-df['Open'])
df['Range'] = 10000 * (df['Low']-df['High'])

In [ ]:
#now we add, the difference between the close price, and the Moving average of closes for various time periods
#start by making a list of most commonly used MAs, plus some that make sense for an hourly timeframe
MA_list = [5,7,9,12,14,21,24,50,100,120,200]
for n in MA_list:
    name = 'MA'+ str(n)
    df[name] = 10000*(df['Close'] - df['Close'].rolling(n).mean())

In [ ]:
#now we do the difference between the current range, and the moving average of hourly ranges for the same lookback periods as above

for n in MA_list:
    name = 'Range'+ str(n)
    df[name] = df['Range'] - df['Range'].rolling(n).mean()

In [ ]:
#And lastly a check if this is the highest high or lowest Low over the last 12 hours
#The code checks if the current high/low is the max of the rolling high/low
for n in range(2,13):
    name = 'HH'+ str(n)
    name2 = 'LL'+ str(n)
    df[name] = df['High'] == df['High'].rolling(n).max()
    df[name2] = df['Low'] == df['Low'].rolling(n).min()

In [ ]:
#Now we create 2 columns which will include 2 potential dependant variables
#In this case, is the  next candle change >=  +10pips, or <= -10 pips 
df['Buy'] = df['Change'].shift(-1) >=10
df['Sell'] = df['Change'].shift(-1) <=-10

#Lastly, we drop the NaN values, and all the columns which don't matter
df = df.dropna()
df = df.drop(columns=['Date_Time','Open','High','Low','Close'])

df.to_csv("EURUSD1H_Cleaned.csv", index=False)